## Demo
Notebook to present the functionality of the EPANETUtils class made by me.

In [ ]:
import json
import time
import pandas as pd

import sys  
sys.path.insert(0, './../utils-py')
from EPANETUtils import EPANETUtils

In [ ]:
file = "./../data/RaduNegru11Jan2021WithDemands_2.2.inp"
sensors = ["SenzorComunarzi-NatVech", "SenzorCernauti-Sebesului", "SenzorChisinau-Titulescu", "SenzorComunarzi-castanului"]
reservoirs_and_closest_junctions = [("GA-Braila", "751-B"), ("Apollo", "748-B"), 
                                    ("RaduNegruMare", "763-B"), ("RaduNegru2", "760-B")]
for reservoir, junction in reservoirs_and_closest_junctions:
    sensors.append(junction)
    
display(sensors)

In [ ]:
print("Starting simulation.....")
now = time.time()
epanet_util_instance_2 = EPANETUtils(file, "PDD")

print("Generating leaks.....")
leakage_arr = [0.0, 0.003, 0.006, 0.012]
dict_leaks2, dict_diff2 = epanet_util_instance_2.run_leakage_scenario(leaks_arr=leakage_arr, generate_diff_dict=True, 
                                                                      retrieve_specific_nodes_arr=sensors)

with open("1-leaks-dict-selected.json", 'w') as fp:
    json.dump(dict_leaks2, fp, indent=4) # 
print("To calculate it took: {}s".format(time.time() - now))

later = time.time()
print("Total duration: {}s".format(later - now))

In [ ]:
# TODO calibrate data to pressures
print("Starting simulation.....")
now = time.time()
epanet_util_instance_1 = EPANETUtils(file, "PDD")

print("Generating leaks.....")
dict_leaks1, dict_diff1 = epanet_util_instance_1.run_leakage_scenario(leaks_arr=[0.006, 0.012], generate_diff_dict=True)

with open("1-leaks-dict-all.json", 'w') as fp:
    json.dump(dict_leaks1, fp, indent=4)
print("To calculate it took: {}s".format(time.time() - now))

later = time.time()
print("Total duration: {}s".format(later - now))

In [ ]:
"""# Files are 130MB smaller if there is no indent
with open("1-leaks-dict.json", 'w') as fp:
    json.dump(dict_leaks, fp, indent=4)

print("Leaks to file it took: {}s".format(time.time() - now))

with open("1-diff-dict.json", 'w') as fp:
    json.dump(dict_diff, fp, indent=4)""";